# Finding Useful Features/Variables/Input

A regression model that has fewer input/independent variables is often desirable for simplicity and explainability. It is often referred to as having a **parsimonius** model. See this article for more details on the "[law of parsimony][1]".

Let's try a few different ways to find the "best" model using our customer dataset.

[1]: https://en.wikipedia.org/wiki/Occam%27s_razor

In [ ]:
# Import our most-used packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Import useful ones from the sklearn package 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## RFE classes
from sklearn.feature_selection import RFE, RFECV

## Other classes for linear models
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV

# Scaling
from sklearn.preprocessing import StandardScaler

# Import statsmodels stuff
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

## The Data
We have been given sample data from our customers. The data has been aggregated from individual purchases / transactions across the time period (e.g., last year). The goal is to see if we can predict the spending amounts for the next time period (e.g., next year).

In [ ]:
# Read in the data and print out its shape
cust = pd.read_csv('./data/customers_clean.csv')
print(cust.shape)

In [ ]:
# Look at info
cust.info()

In [ ]:
# Sample the data
cust.sample(5)

In [ ]:
# See summary statistics
cust.describe()

## End Result for Input
We want to have all numerical variables. This means we should create *dummy* variables for `gender`, `marital_status`, and `home_ownership`. We also do not need `cust_id` since it is just a unique id (now that we have dropped duplicates). The two date columns could be used to create numerical values, but we will simply ignore them for now.

In [ ]:
# Let's drop the following columns:
# cust_id, join_date, last_purchase_date
new_cust = cust.drop(columns=['cust_id','join_date','last_purchase_date'])
new_cust.info()

## Create Dummy Variables

If your `DataFrame` contains categorical (or object) columns, you can call `pd.get_dummies(your_dataframe)` to create the dummy variables for **every** categorical column in the `DataFrame`. Since we deleted the "extra" columns, let's try it on our `new_cust` variable and see the results.

In [ ]:
# Run it again and save it in a new DataFrame
data = pd.get_dummies(new_cust, dtype=int, drop_first=True)
data.info()

In [ ]:
# Look at .describe()
data.describe()

## Split Data into Training and Test Sets

Need to first define `X` and `y`. Then we can try train/test split.

In [ ]:
# define the output variable, y
y = data.spend

# define the X
X = data.drop('spend', axis=1)

In [ ]:
# Time to split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=163)

In [ ]:
# Look at shape of X_train
X_train.shape

In [ ]:
# Look at shape of X_test
X_test.shape

In [ ]:
# Kick out summary statistics for X_train
X_train.describe()

In [ ]:
# Kick out summary statistics for X_test
# Hope these are close to X_train stats
X_test.describe()

In [ ]:
full_train = pd.concat([X_train, y_train], axis='columns')
full_train

In [ ]:
# Run a full regression model with statsmodels
# Create the RHS (the independent variables)
rhs = '+'.join([str(col) for col in X_train.columns])
print(rhs)

In [ ]:
# Full multiple linear regression with statsmodels
full_mlr = ols('spend ~ ' + rhs, data=full_train).fit()

In [ ]:
full_mlr.summary()

## Finding VIFs

Unfortunately, the `summary()` method does not include the VIFs for each variable. (Wouldn't that be nice!) We can, however, find them somewhat painlessly. The code cell below contains a function that will compute VIFs for the features/variables that are passed into it using the `X_train` dataset we created earlier.

In [ ]:
# compute the vif for all given features
def compute_vif(considered_features):
    X = X_train[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

In [ ]:
# Send the function all the columns from X_train
compute_vif(X_train.columns)

### What Does This Mean?

You tell me!

## We Should Scale the Data

Before we attempt to use RFE, Ridge, Lasso, or ElasticNet, we should scale the data. We will use the `StandardScaler` which makes each variable have a mean of 0 and a standard deviation of 1. It does not change the shape of the data. (Thought Exercise: How would you verify that statement?)

In [ ]:
# Fit the scaler on just the training X variables
# Let's start with StandardScaler which will center
# each variable at 0 and give each a unit variance (=1)
s_scaler = StandardScaler().fit(X_train)
s_scaler

In [ ]:
# Transform X_train
X_train_ss = s_scaler.transform(X_train)
# Put it in a DataFrame using same column names
train_X_ss = pd.DataFrame(X_train_ss, columns=X_train.columns)

In [ ]:
# Take a look at the DataFrame
train_X_ss

In [ ]:
# Summary statistics for scaled data
train_X_ss.describe()

## RFE 

Recursive feature elimination will rank the input variables if you tell it to only select a single feature. Let's try it and see what order it thinks the input variables add value to the regression.

In [ ]:
# Run regression on the standardized data (mean=0, stdev=1)
mlr = LinearRegression()
rfe = RFE(mlr, n_features_to_select=1)

mlr.fit(rfe.fit_transform(train_X_ss, y_train), y_train)

In [ ]:
# Print a boolean array called support_
# Because we had n_features_to_select=1, only one will be True
rfe.support_

In [ ]:
# What order are the other input variables in?
rfe.ranking_

In [ ]:
# Let's try to make sense of the ranking by matching each ranking
# with its column name
print('Input variables sorted by their rank:')

d = {}
for i in range(len(rfe.ranking_)):
    d[rfe.ranking_[i]] = train_X_ss.columns[i]
    
print(sorted(d.items()))

### 'Optimal' Number of Features?

Let's try RFECV that is supposed to find the "optimal" number of features for us

In [ ]:
mlr_rfecv = LinearRegression()
rfecv = RFECV(mlr_rfecv)

mlr_rfecv.fit(rfecv.fit_transform(train_X_ss, y_train), y_train)
print(f'Optimal number of features: {rfecv.n_features_}')

In [ ]:
# Look at the support and ranking
print(rfecv.support_)
print(rfecv.ranking_)

In [ ]:
# Print out all the variables that have a ranking of 1
for i in range(len(rfecv.ranking_)):
    if rfecv.ranking_[i] == 1:
        print(f'Ranking of 1: {train_X_ss.columns[i]}')

Notice that those all "tie" for a ranking of 1. It does not tell us which of those 5 input variables is the most important similar to what we saw with `RFE` setting `n_features_to_select=1`. In this case, they are simply printed out in the order they appear in the `DataFrame`.

In [ ]:
# What do the estimated coefficients look like?
rfecv.estimator_.coef_

## Verify Estimated Coefficients

Lets' use `statsmodels` to verify the estimated coefficients that we received from the `RFECV` model. We are going to use a slightly different implementation of ordinary least squares than we have used from `statsmodels` before. The reason is that it you can pass in two different datasets, one for the independent variables and one for the dependent variable which makes our lives a bit simpler.

In [ ]:
# Create a new DataFrame called new_X_train that contains "important" variables
new_X_train = train_X_ss[['household_income','gender_M','marital_status_unmarried','home_ownership_rent','home_ownership_unknown']]

# We need a constant term with the approach we are taking
new_X_train = sm.add_constant(new_X_train)

# See what new_X_train looks like
new_X_train

In [ ]:
# We need to create a DataFrame for the y_train data
# Most notably, we are resetting the index so that it
# aligns with the index for the X variables
train_y = pd.DataFrame(y_train,
             columns=['spend']).reset_index(drop=True)

In [ ]:
# Take a look at it
train_y

In [ ]:
# Create a OLS model, fit it, and look at its summary
m = sm.OLS(train_y, new_X_train).fit()
m.summary()

## Ridge Regression
Let's try Ridge regression (also called the $\ell_{2}$ norm).

In [ ]:
# Create a ridge with alpha=0
# This should create a model with the same
# estimated coefficients as the full model
ridge = Ridge(alpha=0)
ridge.fit(X_train_ss, y_train)

In [ ]:
# Put the estimated coefficients in a Series for easier viewing
pd.Series(ridge.coef_, index=train_X_ss.columns)

### Alpha's Effect

In general, the larger the value of $\alpha$, the more penalization (regularization) which will shrink the estimated coefficients. We may have to change the value of alpha several times to see any major changes in the estimated coefficients.

In [ ]:
# Try changing alpha
ridge2 = Ridge(alpha=4)
ridge2.fit(X_train_ss, y_train)
pd.Series(ridge2.coef_, index=train_X_ss.columns)

In [ ]:
alphas = 10**np.linspace(3,1,100)
alphas

In [ ]:
ridge = Ridge()
coefs = []

for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(X_train_ss, y_train)
    coefs.append(ridge.coef_)

ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
#ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.legend(train_X_ss.columns);

In [ ]:
# Do a CV Ridge Regression to find best alpha
ridgecv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridgecv.fit(X_train_ss, y_train)

In [ ]:
# Best alpha?
ridgecv.alpha_

In [ ]:
# Create a Ridge Regression with the found alpha
rba = Ridge(alpha=ridgecv.alpha_)
rba.fit(X_train_ss, y_train)

In [ ]:
pd.Series(rba.coef_, index=train_X_ss.columns)

## Lasso

Let's try Lasso

In [ ]:
lasso = Lasso(max_iter=10000)
coefs = []

for a in alphas*2:
    lasso.set_params(alpha=a)
    lasso.fit(X_train_ss, y_train)
    coefs.append(lasso.coef_)

ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
#ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Lasso coefficients as a function of the regularization')
plt.legend(train_X_ss.columns);

In [ ]:
# Create a Lasso with default alpha
lasso1 = Lasso()
lasso1.fit(X_train_ss, y_train)
lasso1.coef_

In [ ]:
# LassoCV
lassocv = LassoCV(alphas=alphas, max_iter=10000)
lassocv.fit(X_train_ss, y_train)

In [ ]:
# Best alpha for LassoCV
lassocv.alpha_

In [ ]:
# Coefficients
lassocv.coef_

## ElasticNet

In [ ]:
# Create an ElasticNet using default parameters
en = ElasticNet()
# fit it
en.fit(X_train_ss, y_train)
# Look at the estimated coefficients
en.coef_

In [ ]:
# Create an ElasticNetCV to find best alpha
enCV = ElasticNetCV(cv=5, alphas=alphas, max_iter=10000)
enCV.fit(X_train_ss, y_train)

In [ ]:
# best alpha for elasticnetcv
enCV.alpha_

In [ ]:
# Build ElasticNet with best alpha
en2 = ElasticNet(alpha=enCV.alpha_)
en2.fit(X_train_ss, y_train)
en2.coef_

In [ ]:
# Make predictions with the ElasticNet model en2
en2.predict(s_scaler.transform(X_test))

In [ ]:
# Find the RMSE of that model
np.sqrt(mean_squared_error(y_test, en2.predict(s_scaler.transform(X_test))))

**&copy; 2022 - Present: Matthew D. Dean, Ph.D.   
Clinical Associate Professor of Business Analytics at William \& Mary.**